# YOLOv8 Introduction

This notebook demonstrates how to use YOLOv8 models from Ultralytics for object detection tasks. We will cover:

1. **Inference** - Using pre-trained models to detect objects in images
2. **Fine-tuning** - Adapting a pre-trained model to our PASCAL VOC dataset
3. **Training from scratch** - Explaining the process of creating a new model trained on our dataset

The PASCAL VOC dataset contains 20 object categories including person, car, dog, etc. We'll be using the VOC2007 and VOC2012 datasets which are already downloaded and located at the path specified in the `VOC_ROOT` environment variable.


## Setup

First, let's import the required libraries and set up our environment. We need to ensure the `VOC_ROOT` environment variable is loaded correctly.


In [ ]:
# Check ultralytics version
!pip show ultralytics | grep Version

In [1]:
import os
from pathlib import Path
import random
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
from dotenv import load_dotenv

# Get project root directory
project_root = Path().absolute().parents[3]  # Assumes notebook is in notebooks/model/ext/yolov8
print(f"Project root: {project_root}")

# Add project root to sys.path if needed (optional, depends on execution context)
# if str(project_root) not in sys.path:
#     sys.path.append(str(project_root))

# Load environment variables from .env file at the project root
dotenv_path = project_root / ".env"
loaded = load_dotenv(dotenv_path=dotenv_path)
print(f".env loaded: {loaded}")

# Access VOC_ROOT environment variable
VOC_ROOT = os.environ.get("VOC_ROOT")
if VOC_ROOT:
    print(f"VOC dataset path: {VOC_ROOT}")
    # Check if the directory exists
    if not Path(VOC_ROOT).is_dir():
        print(f"Warning: VOC_ROOT directory '{VOC_ROOT}' does not exist! Please check .env file.")
else:
    print("Error: VOC_ROOT environment variable not set. Please check .env file.")

Project root: /home/wmu/vibe/vibe_coding
.env loaded: True
VOC dataset path: /home/wmu/vibe/hub/datasets/VOC


## 1. Inference with Pre-trained YOLOv8 Models

YOLOv8 comes with several pre-trained models. Let's load `yolo8n.pt` (YOLOv8 Nano) and run inference on a random image from the VOC dataset.

_Note: The first time you use a model like `yolo8n.pt`, Ultralytics will automatically download it._


In [ ]:
# Load a pre-trained YOLOv8 nano model
try:
    model = YOLO("yolov8n.pt")
    print(f"Model yolo8n.pt loaded successfully.")
    # Print model class names (should be COCO classes initially)
    # print("Default model classes:", model.names)
except Exception as e:
    print(f"Error loading model: {e}")

In [ ]:
# Function to load a random image path from the VOC dataset images/test2007 folder
# Ensure necessary imports are available (usually done in earlier cells, but good practice)
from pathlib import Path

def load_random_voc_image(voc_root_path):
    """Loads a random image path from the VOC dataset's images/test2007 directory."""
    if not voc_root_path or not Path(voc_root_path).is_dir():
        print("VOC_ROOT path is invalid or not set.")
        return None

    # Target the test2007 directory within images/
    test_images_dir = Path(voc_root_path) / 'images' / 'test2007'
    if not test_images_dir.is_dir():
        print(f"Test images directory not found at {test_images_dir}")
        return None

    try:
        # Find all JPEG images in the directory using Path.glob
        image_files = list(test_images_dir.glob('*.jpg'))
        # Alternative using glob module:
        # image_files = glob.glob(str(test_images_dir / '*.jpg'))

        if not image_files:
            print(f"No .jpg images found in {test_images_dir}")
            return None

        # Select a random image path
        random_image_path = random.choice(image_files)

        print(f"Selected random image: {random_image_path}")
        return str(random_image_path) # Return as string

    except Exception as e:
        print(f"Error loading random image from {test_images_dir}: {e}")
        return None

# Get a random test image path using the updated function
test_image_path = load_random_voc_image(VOC_ROOT)

In [ ]:
# Perform inference if image path is valid
if test_image_path and "model" in locals():
    try:
        results = model(test_image_path)

        # Display the results
        # results[0].show() # Opens a separate window

        # Plot results within the notebook
        res_plotted = results[0].plot()  # Returns annotated image as numpy array (BGR)
        plt.figure(figsize=(12, 8))
        plt.imshow(
            cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
        )  # Convert BGR to RGB for matplotlib
        plt.axis("off")
        plt.title("YOLOv8 Inference Results")
        plt.show()
    except Exception as e:
        print(f"Error during inference or plotting: {e}")
else:
    print("Skipping inference due to missing image path or model.")

### Examining Detection Results

The `results` object contains detailed information about the detections.


In [ ]:
# Examine the first result if available
if "results" in locals() and results:
    result = results[0]  # First image result

    # Access bounding boxes (xyxy format), confidence scores, and class IDs
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes: x1, y1, x2, y2
    confidences = result.boxes.conf.cpu().numpy()  # Confidence scores
    class_ids = result.boxes.cls.cpu().numpy().astype(int)  # Class IDs

    # Get class names from the model
    class_names = result.names  # Use names from the result object directly

    # Print detection details
    print(f"Found {len(boxes)} objects:\n")
    for i, (box, conf, cls_id) in enumerate(zip(boxes, confidences, class_ids)):
        cls_name = class_names[cls_id]
        print(f"Detection {i + 1}: Class='{cls_name}' ({cls_id}), Confidence={conf:.2f}, Box={box}")
else:
    print("No results available to examine.")

## 2. Fine-tuning YOLOv8 on PASCAL VOC Dataset

Let's fine-tune the pre-trained `yolo8n.pt` model on our PASCAL VOC dataset. We will use the configuration file `src/models/ext/yolov8/configs/voc.yaml`.

**Important:** Fine-tuning requires significant computational resources (GPU recommended) and time. For this demo, we'll run for only **1 epoch** with a **small image size** and **small batch size**. Adjust these parameters (`epochs`, `imgsz`, `batch`) for actual training based on your hardware.


In [2]:
# Path to our VOC dataset configuration YAML
voc_yaml_path = project_root / 'src/models/ext/yolov8/configs/voc2007.yaml'
print(f"Dataset config path: {voc_yaml_path}")

# Check if the YAML file exists
if not voc_yaml_path.is_file():
    print(f"Error: Dataset YAML file not found at {voc_yaml_path}")

Dataset config path: /home/wmu/vibe/vibe_coding/src/models/ext/yolov8/configs/voc2007.yaml


In [3]:
# Proceed only if YAML exists and VOC_ROOT is valid
if voc_yaml_path.is_file() and VOC_ROOT and Path(VOC_ROOT).is_dir():
    try:
        # Create a fresh model instance for fine-tuning
        finetune_model = YOLO("yolov8n.pt")
        print("Loaded yolo8n.pt for fine-tuning.")

        # Define training parameters for the demo
        # !!! Adjust these for real training !!!
        train_results = finetune_model.train(
            data=str(voc_yaml_path),  # Path to the dataset YAML
            epochs=1,  # DEMO ONLY: Use 50-100+ for real training
            imgsz=320,  # DEMO ONLY: Use 640 for real training
            batch=8,  # DEMO ONLY: Increase based on GPU memory (e.g., 16, 32)
            name="yolov8n_voc_finetune_demo",  # Experiment name (results saved to runs/detect/...)
            project="runs/detect",  # Project directory to save runs
            exist_ok=True,  # Overwrite existing experiment folder if needed
            verbose=True,  # Show detailed training progress
            device='0,1'                # Specify GPU device (e.g., 0) or 'cpu'
        )
        print("Fine-tuning demo completed. Results saved to runs/detect/yolov8n_voc_finetune_demo")

        # The best model weights are usually saved as 'best.pt' in the experiment directory
        best_weights_path = Path("runs/detect/yolov8n_voc_finetune_demo/weights/best.pt")
        if best_weights_path.is_file():
            print(f"Best weights saved at: {best_weights_path}")
        else:
            print("Could not find best.pt. Check training output.")

    except Exception as e:
        print(f"Error during fine-tuning: {e}")
else:
    print("Skipping fine-tuning due to missing YAML file or invalid VOC_ROOT.")

Loaded yolo8n.pt for fine-tuning.
Ultralytics 8.3.98 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
                                                      CUDA:1 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/wmu/vibe/vibe_coding/src/models/ext/yolov8/configs/voc2007.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=320, save=True, save_period=-1, cache=False, device=0,1, workers=8, project=runs/detect, name=yolov8n_voc_finetune_demo, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=F

100%|██████████| 755k/755k [00:00<00:00, 1.56MB/s]


Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: muwayne-vibe (muwayne-vibe-dailygpt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.8
wandb: Run data is saved locally in /home/wmu/vibe/vibe_coding/notebooks/model/ext/yolov8/wandb/run-20250329_140400-6la5c9wg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run yolov8n_voc_finetune_demo
wandb: ⭐️ View project at https://wandb.ai/muwayne-vibe-dailygpt/runs-detect
wandb: 🚀 View run at https://wandb.ai/muwayne-vibe-dailygpt/runs-detect/runs/6la5c9wg


Overriding model.yaml nc=80 with nc=20
Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all       2510       7818      0.557      0.308      0.318      0.219

1 epochs completed in 0.017 hours.
Optimizer stripped from runs/detect/yolov8n_voc_finetune_demo/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8n_voc_finetune_demo/weights/best.pt, 6.2MB

Validating runs/detect/yolov8n_voc_finetune_demo/weights/best.pt...
Ultralytics 8.3.98 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
                                                      CUDA:1 (NVIDIA RTX A5000, 24248MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 314/314 [00:11<00:00, 28.53it/s]


                   all       2510       7818      0.561        0.3      0.318      0.219
             aeroplane        127        175      0.429      0.099      0.234      0.169
               bicycle        133        216      0.389      0.421      0.387      0.254
                  bird        151        305      0.289       0.22      0.179      0.129
                  boat        101        190          1          0     0.0127    0.00886
                bottle        109        296          1          0     0.0678      0.043
                   bus         97        141      0.408      0.355      0.345      0.309
                   car        359        818      0.397      0.663      0.588      0.404
                   cat        178        198      0.376      0.652      0.568       0.41
                 chair        290        706      0.331      0.251      0.219      0.105
                   cow         75        171          0          0      0.068     0.0477
           diningtabl

wandb: uploading artifact run_6la5c9wg_model; uploading artifact run-6la5c9wg-curvesF1-ConfidenceB_table; uploading artifact run-6la5c9wg-curvesPrecision-ConfidenceB_table; uploading artifact run-6la5c9wg-curvesRecall-ConfidenceB_table
wandb: uploading artifact run-6la5c9wg-curvesPrecision-ConfidenceB_table; uploading artifact run-6la5c9wg-curvesRecall-ConfidenceB_table
wandb: uploading history steps 0-0, summary, console lines 55-56
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▁
wandb:                  lr/pg1 ▁
wandb:                  lr/pg2 ▁
wandb:        metrics/mAP50(B) ▁
wandb:     metrics/mAP50-95(B) ▁
wandb:    metrics/precision(B) ▁
wandb:       metrics/recall(B) ▁
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss ▁
wandb:          train/cls_loss ▁
wandb:          train/dfl_loss ▁
wandb:            

Fine-tuning demo completed. Results saved to runs/detect/yolov8n_voc_finetune_demo
Best weights saved at: runs/detect/yolov8n_voc_finetune_demo/weights/best.pt


### Evaluating the Fine-tuned Model

After fine-tuning (even for 1 epoch), we can evaluate its performance on the validation set defined in `voc.yaml`.


In [ ]:
# Path to the fine-tuned weights (use 'best.pt' if it exists)
fine_tuned_weights = None
if "best_weights_path" in locals() and best_weights_path.is_file():
    fine_tuned_weights = str(best_weights_path)
elif Path("runs/detect/yolov8n_voc_finetune_demo/weights/last.pt").is_file():
    # Fallback to last epoch weights if best.pt not found
    fine_tuned_weights = "runs/detect/yolov8n_voc_finetune_demo/weights/last.pt"
    print("Using last.pt for evaluation as best.pt was not found.")

if fine_tuned_weights and voc_yaml_path.is_file():
    try:
        # Load the fine-tuned model
        eval_model = YOLO(fine_tuned_weights)
        print(f"Loaded fine-tuned model: {fine_tuned_weights}")

        # Evaluate on the validation set
        # The validation set is specified in voc.yaml
        val_results = eval_model.val(
            data=str(voc_yaml_path),
            split="val",  # Explicitly specify the validation split
            # imgsz=320,             # Can specify image size for validation too
            # batch=8                # Can specify batch size for validation
            name="yolov8n_voc_eval_demo",  # Separate name for evaluation results
        )
        # The results object (val_results.box.map50, etc.) holds the metrics
        print("Validation completed.")
        # print(val_results.box) # Show detailed box metrics

    except Exception as e:
        print(f"Error during evaluation: {e}")
else:
    print("Skipping evaluation due to missing fine-tuned weights or YAML file.")

In [ ]:
# Test the fine-tuned model on a random image
if "eval_model" in locals():
    test_image_path_ft = load_random_voc_image(VOC_ROOT)
    if test_image_path_ft:
        try:
            results_ft = eval_model(test_image_path_ft)

            # Display the results
            res_plotted_ft = results_ft[0].plot()
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(res_plotted_ft, cv2.COLOR_BGR2RGB))
            plt.axis("off")
            plt.title("Fine-tuned YOLOv8 Results (1 Epoch Demo)")
            plt.show()

            # Optionally, print class names (should now be VOC classes)
            # print("Fine-tuned model classes:", eval_model.names)

        except Exception as e:
            print(f"Error during inference with fine-tuned model: {e}")
    else:
        print("Could not load random image for fine-tuned test.")
else:
    print("Skipping inference with fine-tuned model as it wasn't loaded/trained.")

## 3. Training YOLOv8 from Scratch (Explanation)

Training a YOLOv8 model from scratch involves initializing a model with random weights based on an architecture definition (e.g., `yolo8n.yaml`) instead of loading pre-trained weights (`yolo8n.pt`).

The process is similar to fine-tuning, but you specify the model architecture YAML instead of a `.pt` file.

```python
# Example code structure (DO NOT RUN unless you intend to train from scratch)

# # Create a new model from the yolo8n architecture YAML
# scratch_model = YOLO('yolo8n.yaml')
#
# # Train from scratch on the VOC dataset
# results_scratch = scratch_model.train(
#     data=str(voc_yaml_path),      # Path to your dataset config
#     epochs=100,                   # Need significantly more epochs (e.g., 100-300)
#     imgsz=640,                    # Standard image size
#     batch=16,                     # Adjust based on GPU memory
#     name='yolov8n_voc_scratch',   # Experiment name
#     project='runs/detect',        # Project directory
#     exist_ok=True,
#     verbose=True
# )
```

**Key Differences & Considerations for Training from Scratch:**

1.  **Initialization:** Starts with `YOLO('yolo8n.yaml')` instead of `YOLO('yolo8n.pt')`. The `.yaml` file defines the network layers and structure. Ultralytics provides standard architecture YAMLs.
2.  **Training Time:** Requires significantly more training epochs (e.g., 100-300+) and overall time compared to fine-tuning.
3.  **Data Requirement:** Generally needs more data to converge well compared to fine-tuning.
4.  **Performance:** May not achieve the same peak performance as fine-tuning a model pre-trained on a large, diverse dataset like COCO, especially if your target dataset is small.
5.  **Use Cases:** Useful when your target domain or required architecture differs significantly from the pre-trained models, or when you want to build a model without relying on external pre-trained weights.


## Conclusion

This notebook covered the basics of using Ultralytics YOLOv8:

1. Running inference with a pre-trained model (`yolo8n.pt`).
2. Fine-tuning a pre-trained model on the PASCAL VOC dataset using a configuration YAML.
3. Understanding the process for training a model from scratch.

Remember to adjust training parameters like `epochs`, `imgsz`, and `batch` based on your specific needs and hardware capabilities. For more advanced features, consult the [Ultralytics YOLOv8 Documentation](https://docs.ultralytics.com/).
